Replica of Google Colab NB that runs training.\
__Requires mask-rcnn to run__

In [28]:
import sys
sys.path.append('..')

In [30]:
import os
import json
from numpy import zeros, asarray

import mrcnn.utils
import mrcnn.config
import mrcnn.model


In [32]:
# create class and for Dataset, masks, config
class SolarDataset(mrcnn.utils.Dataset):
    def load_dataset(self, dataset_dir, dataset_type='train'):
        """
        Load the dataset from the specified directory and dataset type (train, val, test).
        """
        # Register the classes
        self.add_class("dataset", 1, "solar_panel")
        self.add_class("dataset", 2, "soil")

        assert dataset_type in ['train', 'val', 'test'], "dataset_type must be one of: train, val, test"

        base_dir = os.path.join(dataset_dir, dataset_type)

        images_dir = os.path.join(base_dir, 'images')
        annotations_dir = os.path.join(base_dir, 'annots')

        # Iterate through all files in the images directory
        for filename in os.listdir(images_dir):
            # Image ID is the filename without the file extension
            image_id = filename[:-4]

            img_path = os.path.join(images_dir, filename)
            ann_path = os.path.join(annotations_dir, f"{image_id}_bboxes.json")

            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    def load_mask(self, image_id):
        """
        Generate masks for each instance in the image based on JSON annotations.
        """
        info = self.image_info[image_id]
        path = info['annotation']
        boxes, class_ids, h, w = self.extract_boxes(path)

        masks = zeros([h, w, len(boxes)], dtype='uint8')

        for i, box in enumerate(boxes):
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1

        return masks, asarray(class_ids, dtype='int32')

    def extract_boxes(self, filename):
        """
        Extract bounding boxes and class IDs from JSON annotation file.
        """
        with open(filename) as f:
            data = json.load(f)

        image_dimensions = data["image_dimensions"]
        height, width = image_dimensions["height"], image_dimensions["width"]

        boxes = []
        class_ids = []
        for item in data["objects"]:
            label = item['label']
            if label in ['solar_panel', 'soil']:
                bbox = item['bbox']
                boxes.append([bbox[0], bbox[1], bbox[2], bbox[3]])
                class_ids.append(self.class_names.index(label))

        return boxes, class_ids, height, width

class SolarConfig(mrcnn.config.Config):
  NUM_CLASSES = 1 + 2
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  def __init__(self):
      super().__init__()
      self.train_images_dir = '../RCNN_Masks/train/images'
      self.NAME = "solar_cfg"
      self.num_train_images = len([name for name in os.listdir(self.train_images_dir) if os.path.isfile(os.path.join(self.train_images_dir, name))])
      self.STEPS_PER_EPOCH = self.num_train_images // self.IMAGES_PER_GPU  # Adjust based on your actual number of images in the training set

In [33]:
# Train
train_dataset = SolarDataset()
train_dataset.load_dataset(dataset_dir='../RCNN_Masks/', dataset_type='train')
train_dataset.prepare()

In [34]:
# Validation
validation_dataset = SolarDataset()
validation_dataset.load_dataset(dataset_dir='../RCNN_Masks/', dataset_type='val')
validation_dataset.prepare()

# Model Configuration
solar_config = SolarConfig()
solar_config.display()

In [35]:
solar_config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        35
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      0
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                13
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [36]:
# Build the Mask R-CNN Model Architecture
model = mrcnn.model.MaskRCNN(mode='training', model_dir='./', config=solar_config)

__Training Cycle on Heads__

In [ ]:
model.load_weights(filepath='mask_rcnn_coco.h5', by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

model.train(train_dataset=train_dataset,
            val_dataset=validation_dataset,
            learning_rate=solar_config.LEARNING_RATE,
            epochs=1,
            layers='heads')

model_path = 'Solar_mask_rcnn_trained.h5'
model.keras_model.save_weights(model_path)

__Full Training Cycle__

In [ ]:
model.train(train_dataset=train_dataset,
            val_dataset=validation_dataset,
            learning_rate=solar_config.LEARNING_RATE / 10,
            epochs=2,
            layers='all')

model_path = 'Solar_mask_rcnn_trained_all.h5'
model.keras_model.save_weights(model_path)